In [ ]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")




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


In [7]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000234A4AF8110>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000234A4AFA7D0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [9]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Prasanna and I am a learning AI")])

AIMessage(content="Hello Prasanna, it's nice to meet you!\n\nIt's fascinating to meet another AI on the learning path. What kind of things are you learning about? \n\nI'm always eager to learn from others and expand my knowledge. Perhaps we can learn from each other?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 22, 'total_tokens': 84, 'completion_time': 0.112727273, 'prompt_time': 0.000182979, 'queue_time': 0.055342756, 'total_time': 0.112910252}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e3559e38-7944-4ef8-8fcc-d98adf2ee12e-0', usage_metadata={'input_tokens': 22, 'output_tokens': 62, 'total_tokens': 84})

In [10]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Prasanna and I am a learning AI"),
        AIMessage(content="Hello Prasanna, it's nice to meet you!\n\nIt's fascinating to meet another AI on the learning path. What kind of things are you learning about? \n\nI'm always eager to learn from others and expand my knowledge. Perhaps we can learn from each other?\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You said your name is Prasanna, and you are a learning AI. 😊 \n\nIs there anything else you'd like me to know about you?  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 104, 'total_tokens': 140, 'completion_time': 0.065454545, 'prompt_time': 0.003538063, 'queue_time': 0.052805482, 'total_time': 0.068992608}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-7189680d-d835-4e33-8f03-ddc251a07df7-0', usage_metadata={'input_tokens': 104, 'output_tokens': 36, 'total_tokens': 140})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

This function checks whether a chat history already exists for a given session_id:
If it exists, it returns the existing history.
If not, it creates a new ChatMessageHistory instance and stores it in store.
This ensures that each session has persistent chat history.



ChatMessageHistory: Provides a simple chat history storage mechanism.
BaseChatMessageHistory: An abstract base class for managing chat history.
RunnableWithMessageHistory: Wraps a model to include chat history retrieval, so the model can maintain context between interactions.

In [12]:
config={"configurable":{"session_id":"chat1"}}

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Prasanna and I am a learning AI")],
    config=config
)

In [14]:
response.content

"Hello Prasanna! It's nice to meet you. \n\nI'm glad to hear you're learning about AI. It's a fascinating field!  \n\nWhat are you most interested in learning about? Maybe I can help you explore some concepts or answer any questions you have.\n"

In [15]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Prasanna. 😊  \n\nI remember that from our introduction.  Is there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 99, 'total_tokens': 129, 'completion_time': 0.054545455, 'prompt_time': 0.002550465, 'queue_time': 0.052415207, 'total_time': 0.05709592}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-4366ff3f-27b7-4648-b4c0-8f856f2198b2-0', usage_metadata={'input_tokens': 99, 'output_tokens': 30, 'total_tokens': 129})

In [16]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to use it! 😊\n"

In [17]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John! It's nice to meet you.  How can I help you today? 😊\n"

In [18]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John,  you told me earlier! 😊 \n\nIs there anything else I can help you with, John?\n'